In [1]:
#!/usr/bin/env python
# vim:fileencoding=utf-8
import sklearn
print(sklearn.__version__)

import sys
import numpy as np
import pandas as pd

#音声信号処理
import scipy.signal as sg
import soundfile as sf

#スケール変換
from sklearn.preprocessing import MinMaxScaler

#NMF
from sklearn.decomposition import NMF

#可視化ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns

0.18.1


In [2]:
#音楽ファイル
Wav_file = './Input/01_Radio/NHKRadio_pinknoise.wav'
FileName = 'NHKRadio_pinknoise'

#パラメータ
n_components_num = 50

In [3]:
plt.close("all")

# wavファイル読み込み
Wav_wav, Wav_fs = sf.read(Wav_file)

In [ ]:
#取得した音声ファイル
X = Wav_wav[:int(Wav_fs * 10)]

#1.MinMaxScaler（広がりを補正、非負値化）
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
#NMFの行列分解
#model = NMF(n_components=n_components_num, random_state=10, beta_loss='kullback-leibler', solver='mu')

#2020年6月3日にデグレーションが発生して、
#Version[0.18.1]のメソッドに変更
#https://scikit-learn.org/0.18/modules/generated/sklearn.decomposition.NMF.html
model = NMF(n_components=n_components_num, random_state=10, beta=2)
W = model.fit_transform(X_scaled)
H = model.components_

In [ ]:
print(H.shape)
print(W.shape)

In [ ]:
dataframe_W = pd.DataFrame(W)
dataframe_W

In [ ]:
#係数行列の合計値の係数値でソートして、
#係数値上位10個の列番号を取得
W_Top10_index = dataframe_W.mean().sort_values(0, ascending=False).head(10).index
W_Top10_index

In [ ]:
#重要な基底の列番号のみ係数行列を抽出
dataframe_W_index_matrix = dataframe_W[W_Top10_index]
dataframe_W_index_matrix

In [ ]:
#変換特徴量の基底数を指定
dataframe_write_list = list(range(n_components_num))

#重要な基底の列番号だけ繰り返す
for index_value in W_Top10_index:
    
    #指定したindexを除去
    #重要度が高い基底の列番号を除去
    dataframe_write_list.remove(index_value)

In [ ]:
#(信号再構成処理用の行列作成)
#値が設定されている列でない列を指定して、
#行列の値に0を設定
for dataframe_write_index in dataframe_write_list:
    dataframe_W_index_matrix[dataframe_write_index] = 0
    dataframe_W_index_matrix

In [ ]:
#列番号でソート
dataframe_W_index_matrix = dataframe_W_index_matrix.sort_index(axis='columns')
dataframe_W_index_matrix

In [ ]:
#print(W.shape)
print(dataframe_W_index_matrix.shape)
print(H.shape)

In [ ]:
#WH
#https://numpy.org/doc/stable/reference/generated/numpy.dot.html
WH_write = np.dot(dataframe_W_index_matrix, H)
WH_write

In [ ]:
#信号再構成処理したファイルを出力
sf.write('./Output/Signal_Reconstruction_0.18.1/' + FileName + 'WH_write_Matrix_' + str(n_components_num) + '.wav', Wav_wav, Wav_fs)